In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [14]:
device = 'cpu'

In [15]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [16]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [26]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.1, line_search_method='backtrack')

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.13202518224716187
epoch:  1, loss: 0.11094743758440018
epoch:  2, loss: 0.10617230832576752
epoch:  3, loss: 0.0832819864153862
epoch:  4, loss: 0.07656904309988022
epoch:  5, loss: 0.07209958136081696
epoch:  6, loss: 0.0697375163435936
epoch:  7, loss: 0.06518661975860596
epoch:  8, loss: 0.06255731731653214
epoch:  9, loss: 0.061671335250139236
epoch:  10, loss: 0.061360765248537064
epoch:  11, loss: 0.05896953120827675
epoch:  12, loss: 0.05733287334442139
epoch:  13, loss: 0.05582023039460182
epoch:  14, loss: 0.05564546212553978
epoch:  15, loss: 0.05479069799184799
epoch:  16, loss: 0.053807780146598816
epoch:  17, loss: 0.05325029045343399
epoch:  18, loss: 0.05275009199976921
epoch:  19, loss: 0.05250280350446701
epoch:  20, loss: 0.05248160660266876
epoch:  21, loss: 0.05153047665953636
epoch:  22, loss: 0.051360055804252625
epoch:  23, loss: 0.05064285174012184
epoch:  24, loss: 0.050195902585983276
epoch:  25, loss: 0.04992435500025749
epoch:  26, loss: 0

In [24]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method='backtrack')

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.28115880489349365
epoch:  1, loss: 0.20888052880764008
epoch:  2, loss: 0.17400911450386047
epoch:  3, loss: 0.17035219073295593
epoch:  4, loss: 0.1455872505903244
epoch:  5, loss: 0.12844522297382355
epoch:  6, loss: 0.12219702452421188
epoch:  7, loss: 0.10740651190280914
epoch:  8, loss: 0.09864925593137741
epoch:  9, loss: 0.09577640891075134
epoch:  10, loss: 0.09394504129886627
epoch:  11, loss: 0.08519385010004044
epoch:  12, loss: 0.07421094924211502
epoch:  13, loss: 0.06746806204319
epoch:  14, loss: 0.06649968773126602
epoch:  15, loss: 0.06016349792480469
epoch:  16, loss: 0.05801587551832199
epoch:  17, loss: 0.05683813616633415
epoch:  18, loss: 0.05558540299534798
epoch:  19, loss: 0.052858076989650726
epoch:  20, loss: 0.05063335970044136
epoch:  21, loss: 0.04964904859662056
epoch:  22, loss: 0.048448000103235245
epoch:  23, loss: 0.04830684885382652
epoch:  24, loss: 0.04705002158880234
epoch:  25, loss: 0.04650012403726578
epoch:  26, loss: 0.0457